<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


In [4]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [10]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for lat, lng, label in zip(launch_sites_df.Lat, launch_sites_df.Long, launch_sites_df["Launch Site"]):
    coordinate =  [lat, lng]
    circle = folium.Circle(coordinate, radius=100, color='#d35400', fill=True).add_child(folium.Popup(label))
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % label,
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

In [11]:
# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [12]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [13]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [14]:
launch_sites_df

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


In [15]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

In [16]:
def func(item):
    if item == 1:
        return 'green'
    else:
        return 'red'

launch_sites_df["marker_color"] = launch_sites_df["class"].apply(func)
launch_sites_df

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS SLC-40,28.563197,-80.576820,1,green
2,KSC LC-39A,28.573255,-80.646895,1,green
3,VAFB SLC-4E,34.632834,-120.610745,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [17]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [18]:
def func(item):
    if item == 1:
        return 'green'
    else:
        return 'red'

spacex_df["marker_color"] = spacex_df["class"].apply(func)
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [19]:
from folium import plugins
# Add marker_cluster to current site_map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
launches=plugins.MarkerCluster().add_to(site_map)


# for each row in spacex_df data frame
# create a Marker object with its coordinate

for lat, lng, class1, label in zip(spacex_df.Lat, spacex_df.Long, spacex_df["class"], spacex_df["Launch Site"]):
    coordinate =  [lat, lng]
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % label,
            )
        )
    if class1 == 1:
        fill_color='green'
    else:
        fill_color='red'

    folium.features.CircleMarker(
        coordinate,
        radius=8, # define how big you want the circle markers to be
        color='yellow', # set the circle color based on the first condition
        fill=True,
        popup=label,
        fill_color=fill_color, # set the circle fill color based on the second condition
        fill_opacity=0.6
    ).add_to(launches)
    
    site_map.add_child(marker)    
    site_map.add_child(circle)    

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [20]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [21]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [22]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [23]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [24]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

In [25]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [34]:
coastline_lat_KSC=28.57653
coastline_lon_KSC=-80.607
city_lat_KSC=28.61105
city_lon_KSC=-80.81304
highway_lat_KSC=28.57292
highway_lon_KSC=-80.65541
railway_lat_KSC=28.57304
railway_lon_KSC=-80.65396
launch_site_lat_KSC=28.5731
launch_site_lon_KSC=-80.64737

In [37]:
coastline_lat_CCAFC=28.56281
coastline_lon_CCAFC=-80.56775
city_lat_CCAFC=28.40348
city_lon_CCAFC=-80.60339
highway_lat_CCAFC=28.5624
highway_lon_CCAFC=-80.57071
railway_lat_CCAFC=28.56232
railway_lon_CCAFC=-80.58698
launch_site_lat_CCAFC=28.56244
launch_site_lon_CCAFC=-80.57724

In [ ]:
coastline_lat_VAFB=34.63476
coastline_lon_VAFB=-120.62508
city_lat_VAFB=34.64338
city_lon_VAFB=-120.45753
highway_lat_VAFB=34.63534
highway_lon_VAFB=-120.6231
railway_lat_VAFB=34.63623
railway_lon_VAFB=-120.62379
launch_site_lat_VAFB=34.6325
launch_site_lon_VAFB=-120.61006

In [38]:
distance_coastline_VAFB = calculate_distance(launch_site_lat_VAFB, launch_site_lon_VAFB, coastline_lat_VAFB, coastline_lon_VAFB)
distance_city_VAFB = calculate_distance(launch_site_lat_VAFB, launch_site_lon_VAFB, city_lat_VAFB, city_lon_VAFB)
distance_highway_VAFB = calculate_distance(launch_site_lat_VAFB, launch_site_lon_VAFB, highway_lat_VAFB, highway_lon_VAFB)
distance_railway_VAFB = calculate_distance(launch_site_lat_VAFB, launch_site_lon_VAFB, railway_lat_VAFB, railway_lon_VAFB)

In [39]:
distance_coastline_CCAFC = calculate_distance(launch_site_lat_CCAFC, launch_site_lon_CCAFC, coastline_lat_CCAFC, coastline_lon_CCAFC)
distance_city_CCAFC = calculate_distance(launch_site_lat_CCAFC, launch_site_lon_CCAFC, city_lat_CCAFC, city_lon_CCAFC)
distance_highway_CCAFC = calculate_distance(launch_site_lat_CCAFC, launch_site_lon_CCAFC, highway_lat_CCAFC, highway_lon_CCAFC)
distance_railway_CCAFC = calculate_distance(launch_site_lat_CCAFC, launch_site_lon_CCAFC, railway_lat_CCAFC, railway_lon_CCAFC)

In [40]:
distance_coastline_KSC = calculate_distance(launch_site_lat_KSC, launch_site_lon_KSC, coastline_lat_KSC, coastline_lon_KSC)
distance_city_KSC = calculate_distance(launch_site_lat_KSC, launch_site_lon_KSC, city_lat_KSC, city_lon_KSC)
distance_highway_KSC = calculate_distance(launch_site_lat_KSC, launch_site_lon_KSC, highway_lat_KSC, highway_lon_KSC)
distance_railway_KSC = calculate_distance(launch_site_lat_KSC, launch_site_lon_KSC, railway_lat_KSC, railway_lon_KSC)

In [36]:
print(distance_coastline_VAFB,distance_city_VAFB,distance_highway_VAFB,distance_railway_VAFB)

1.3974297650806125 14.011215189489365 1.2345196215794134 1.323283284321538


In [41]:
print(distance_coastline_KSC,distance_city_KSC,distance_highway_KSC,distance_railway_KSC)

3.961806572527898 16.721759586967263 0.7856262450919639 0.6437650044800987


In [42]:
print(distance_coastline_CCAFC,distance_city_CCAFC,distance_highway_CCAFC,distance_railway_CCAFC)

0.9280158916531832 17.864972849819317 0.6379495114538583 0.9516218641695255


In [26]:
launch_site_marker = folium.Marker(location=[launch_site_lat_VAFB,launch_site_lon_VAFB])
site_map.add_child(launch_site_marker)

# Create a marker at the coastline point with distance information
distance_marker = folium.Marker(
    [coastline_lat_VAFB,coastline_lon_VAFB],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline_VAFB)
    )
)
site_map.add_child(distance_marker)

# Display the map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [66]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)


In [43]:
launch_site_location=[launch_site_lat_VAFB,launch_site_lon_VAFB]
coastline_location_VAFB=[coastline_lat_VAFB,coastline_lon_VAFB]

lines=folium.PolyLine([launch_site_location,coastline_location_VAFB], weight=1)
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [49]:
# Define the coordinates of the launch site, coastline, city, highway, and railway
launch_site_location = [launch_site_lat_VAFB, launch_site_lon_VAFB]
coastline_location_VAFB = [coastline_lat_VAFB, coastline_lon_VAFB]
city_location_VAFB = [city_lat_VAFB, city_lon_VAFB]
highway_location_VAFB = [highway_lat_VAFB, highway_lon_VAFB]
railway_location_VAFB = [railway_lat_VAFB, railway_lon_VAFB]

# Create a map centered at the launch site
site_map = folium.Map(location=launch_site_location, zoom_start=12)

# Create a marker at the launch site
launch_site_marker = folium.Marker(launch_site_location,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'VAFB SLC-4E',
            )
                                  )
site_map.add_child(launch_site_marker)

# Create markers for the coastline, city, highway, and railway
coastline_marker = folium.Marker(coastline_location_VAFB,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline_VAFB)
    )
                                )
city_marker = folium.Marker(city_location_VAFB,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city_VAFB)
    )
                                )
highway_marker = folium.Marker(highway_location_VAFB,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway_VAFB)
    )
                                )
railway_marker = folium.Marker(railway_location_VAFB,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway_VAFB)
    )
                              )
# Add the markers to the map
site_map.add_child(coastline_marker)
site_map.add_child(city_marker)
site_map.add_child(highway_marker)
site_map.add_child(railway_marker)

# Create lines for the coastline, city, highway, and railway
coastline_line = folium.PolyLine([launch_site_location, coastline_location_VAFB], weight=1, color='blue')
city_line = folium.PolyLine([launch_site_location, city_location_VAFB], weight=1, color='red')
highway_line = folium.PolyLine([launch_site_location, highway_location_VAFB], weight=1, color='green')
railway_line = folium.PolyLine([launch_site_location, railway_location_VAFB], weight=1, color='purple')

# Add the lines to the map
site_map.add_child(coastline_line)
site_map.add_child(city_line)
site_map.add_child(highway_line)
site_map.add_child(railway_line)

# Display the map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01)


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
